In [1]:
!pip install bertopic[flair,gensim,spacy,use]

In [2]:
!pip install bertopic[vision]

In [3]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import csv
import re
import nltk

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('portuguese'))
    stop_words.update(["nº","cep","telefone","rua","avenida","endereço","fax","fones"])
    stop_words.update(["egrégia","egrégio","eg","e.g."])
    stop_words.update(["copy","reg","trade","ldquo","rdquo","lsquo","rsquo","bull","middot","sdot","ndash","mdash","cent","pound","euro","ne","frac12","frac14","frac34","deg","larr","rarr","uarr","darr","egrave","eacute","ccedil","hellip"])
    tokens = nltk.word_tokenize(text, language='portuguese')
    tokens_cleaned = [token for token in tokens if token not in stop_words]
    text_cleaned = ' '.join(tokens_cleaned)
    return text_cleaned

In [6]:
def clean_text(doc):
    final_doc = ""
    doc = doc.lower()
    #Tenta identificar parte relevante do documento
    match = re.search(r' cabimento[^\n]*',doc)
    #Palavras irrelevantes
    match_pattern = [r'https?://\S+',r'www\.\S+',r'\S+@\S+',r'\d{2}/\d{2}/\d{4}[ ,]',r'procuradoria regional (federal|da união) da \d+[ªa] região',r'tribunal regional federal[ da] \d+[ªa] região',r'advocacia[ -]geral da união',r'(excelentíssimo|senhor|vice-presidente|desembargador|\(a\))',r'procuradoria[ -]geral federal',r'escritório de advocacia',r'[ superior] tribunal de justiça',r'supremo tribunal federal']
    subs = [''] * len(match_pattern)
    if match:
        start = match.start()
        final_doc = doc[start:]
    else:
        final_doc = doc
    for match_pattern, subs in zip(match_pattern,subs):
        final_doc = re.sub(match_pattern,subs,final_doc)
    final_doc = remove_stopwords(final_doc)
        
    return final_doc
    

In [7]:
#Lê recursos especiais
resp = pd.read_csv('REsp_completo.csv')

docs = []
num_cadastrado = []
indice = []
clean = True

for i, linha in resp.iterrows():
  tipo = type(linha['recurso'])
  try:
    #tratamento necessário, textos estavam sendo identificados como float
    if (tipo == str):
      if clean:
          doc_cleaned = clean_text(linha['recurso'])
          num_cadastrado.append(int(linha['num_tema_cadastrado']))
          docs.append(doc_cleaned)
          indice.append(i)     
      else:
          num_cadastrado.append(int(linha['num_tema_cadastrado']))
          docs.append(linha['recurso'])
          indice.append(i)
      
  except Exception as erro:
    print(f"Erro ao capturar numero de tema cadastrado {i}")
    continue

In [8]:
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

In [9]:
#Cria lista de temas apartir de arquivo
temas_repetitivos_eproc = pd.read_csv('temas_repetitivos.csv', sep=',' )
temas = temas_repetitivos_eproc[['tema','num_tema_cadastrado']].copy()
temas.columns = ['texto','numTema']
list_temas = list(temas.itertuples(index=False, name=None))

In [10]:
temas_seed_list = []
temas_seed = temas_repetitivos_eproc[['tema']].copy()
temas_seed.columns = ['texto']
for indice,linha in temas_seed.iterrows():
    seed = clean_text(linha[0])
    seed = remove_punctuation(seed)
    temas_seed_list.append(seed.split())



In [11]:
#print(temas_seed_list)

In [ ]:

len(docs)
print(docs[470])

In [15]:
len(num_cadastrado)

8186

In [16]:
#Carrega modelo pré-treinado pra criar embeddings dos textos 
#Modelo do bertopic não estava gerando corretamente topicos em portugues estava excluindo letras acentuadas e cedilha
sentence_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
topic_model = BERTopic(embedding_model=sentence_model,top_n_words=13, seed_topic_list=temas_seed_list)
topics, probs = topic_model.fit_transform(docs)

In [17]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,820,-1_pis_cofins_receita_lei,"[pis, cofins, receita, lei, icms, base, cálcul...",[cabimento recurso especial constituição repub...
1,0,864,0_reexame_valor_teto_mínimos,"[reexame, valor, teto, mínimos, salários, obri...","[cabimento recurso art . 1.029 cpc , inciso ii..."
2,1,623,1_496_remessa_necessária_cpc,"[496, remessa, necessária, cpc, 475, ilíquida,...",[cabimento cuida-se julgado negou transito rem...
3,2,153,2_22_indenizado_férias_aviso,"[22, indenizado, férias, aviso, prévio, contri...","[cabimento . conforme mencionado , acórdão ora..."
4,3,137,3_teto_benefício_valor_salário,"[teto, benefício, valor, salário, mvt, menor, ...","[cabimento recurso art . 1.029 cpc , inciso ii..."
...,...,...,...,...,...
203,202,11,202_coisa_julgada_transitado_executivo,"[coisa, julgada, transitado, executivo, segura...",[1 exmo . sr. dr. federal ? tribunal regional ...
204,203,11,203_pena_296_interceptações_daniele,"[pena, 296, interceptações, daniele, fração, c...","[cabimento . presente recurso cabível , vista ..."
205,204,11,204_aquisição_custo_icms_st,"[aquisição, custo, icms, st, mercadoria, subst...","[cabimento recurso especial artigo 105 , iii ,..."
206,205,11,205_teto_menor_benefício_readequação,"[teto, menor, benefício, readequação, salário,...",[tribunal regional federal 2ª região processo ...


In [18]:
representacao = topic_model.get_document_info(docs)

In [19]:
print(representacao)


                                               Document  Topic  \
0     cabimento recurso análise autos , verifica-se ...     40   
1     cabimento recurso especial constituição federa...     -1   
2     cabimento recurso análise autos , verifica-se ...     40   
3     cabimento recurso análise autos , verifica-se ...     40   
4     cabimento recurso análise autos , verifica-se ...     40   
...                                                 ...    ...   
8181  cabimento : ) violação art . 1.022 , ii cpc ex...    136   
8182  cabimento : ) violação art . 1.022 , ii cpc ex...    136   
8183  cabimento : ) violação art . 1.022 , ii cpc ex...    136   
8184  cabimento : ) violação art . 1.022 , ii cpc ex...    136   
8185  cabimento presente recurso especial acórdão qu...     85   

                                              Name  \
0                  40_pasep_banco_diretor_conselho   
1                        -1_pis_cofins_receita_lei   
2                  40_pasep_banco_diretor_con

In [20]:
bertopic_unsupervised = pd.DataFrame()

In [21]:
bertopic_unsupervised["indice"]=indice
bertopic_unsupervised["num_tema_cadastrado"]=num_cadastrado
bertopic_unsupervised["recurso"]=docs
bertopic_unsupervised["topicos"]=representacao['Top_n_words']

In [22]:
print(bertopic_unsupervised)

      indice  num_tema_cadastrado  \
0        NaN                    9   
1        NaN                    9   
2        NaN                    9   
3        NaN                    9   
4        NaN                    9   
...      ...                  ...   
8181     NaN                  163   
8182     NaN                  163   
8183     NaN                  163   
8184     NaN                  163   
8185     NaN                  225   

                                                recurso  \
0     cabimento recurso análise autos , verifica-se ...   
1     cabimento recurso especial constituição federa...   
2     cabimento recurso análise autos , verifica-se ...   
3     cabimento recurso análise autos , verifica-se ...   
4     cabimento recurso análise autos , verifica-se ...   
...                                                 ...   
8181  cabimento : ) violação art . 1.022 , ii cpc ex...   
8182  cabimento : ) violação art . 1.022 , ii cpc ex...   
8183  cabimento : ) viol

In [23]:
#Salva em arquivo os textos com topicos principais extraidos e numeros de temas reais cadastrados
file = 'bertopic_nao_supervisionado.csv'
bertopic_unsupervised.to_csv(file,index=False)

In [24]:
def read_text(dataframe, linha, coluna):
    texto = dataframe.at[linha, coluna]
    return texto

In [25]:
def sort_list(lista):
    return(sorted(lista, key = lambda x: x[1],reverse=True))

In [26]:

def calc_similarity(topics, temas ,k,tema_real):
  lista_similaridade = []
  lista_tema_real = []
  
  for indice, tupla_num_tema in enumerate(temas):
      query_embedding = sentence_model.encode(topics)
      #print(f"Tema {indice} : {tupla_num_tema[0]}")
      tema_cleaned = clean_text(tupla_num_tema[0])
      text_embedding = sentence_model.encode(tema_cleaned)
      tensor_similaridade = util.cos_sim(query_embedding, text_embedding)
      valor_similaridade = tensor_similaridade.item()
      tupla = (tupla_num_tema[1],valor_similaridade)
      lista_similaridade.append(tupla)

  sorted_list = sort_list(lista_similaridade)
  for i, linha in enumerate(sorted_list):
      if(linha[0]==tema_real):
        lista_tema_real.append(i+1)#identifica posição do tema real no ranking
        lista_tema_real.append(linha[1]) 
        break
  ranking = sorted_list[:k]
  return ranking,lista_tema_real

In [27]:
def create_columns(k):
  #k refere-se ao numero de elementos no ranking
  colunas = []
  colunas.append("indice")
  colunas.append("num_tema_cadastrado")
  for i in range(1, k + 1):
    nome = f"sugerido_{i}"
    colunas.append(nome)
    nome = f"similaridade_{i}"
    colunas.append(nome)
  colunas.append("posicao_tema_real")
  colunas.append("similaridade_tema_real")
  return colunas


In [28]:
#Cria dataframe pra armazenar dados sobre textos classificados
nomes_colunas = create_columns(6)
resp_classificados = pd.DataFrame(columns=nomes_colunas)
#Cria arquivo pra armazenar resultados
#resp_classificados.to_csv('resp_classif_bertopic_unsuperv.csv', index=False)
resp_classificados.to_csv('resp_bertopic_unsuperv_filter.csv', index=False)


In [ ]:
#passa topicos de cada documento e lista de temas
for indice, linha in bertopic_unsupervised.iterrows():
  dados = []
  dados.append(indice)
  #numero do tema cadastrado por um analista
  try:
      dados.append(int(linha['num_tema_cadastrado']))
  except Exception as erro:
      print(f"Erro ao capturar numero de tema cadastrado {indice}")
      continue
  
  try:
    #retira hifen entre as palavras dos topicos
      topicos = linha['topicos'].replace("-","")    
      ranking , lista_tema_real = calc_similarity(topicos, list_temas, 6, linha['num_tema_cadastrado'])
  except Exception as erro:
      print(f"Erro calculo similaridade indice {indice}") 
      continue
  for i, tupla_num_tema in enumerate(ranking):
    #captura numero do tema sugerido e valor da similaridade
    dados.append(tupla_num_tema[0])
    dados.append(tupla_num_tema[1])
  
  if(lista_tema_real):
    dados.append(lista_tema_real[0])
    dados.append(lista_tema_real[1])
  else:
    dados.append("NA")
    dados.append("NA")
  #with open('resp_classif_bertopic_unsuperv.csv', mode='a', newline='') as arquivo:
  with open('resp_bertopic_unsuperv_filter.csv', mode='a', newline='') as arquivo:
    writer = csv.writer(arquivo)
    writer.writerow(dados)

In [ ]:
#Salva modelo
topic_model.save("is_notes/my_model_bertopic.pt", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)
